- using heasoft `heasoft-6.31.1 on sciserver`

In [ ]:
from helpers import *
%load_ext autoreload
%autoreload 2

In [ ]:
# set some variable
wdir = '/home/idies/workspace/Temporary/jaclar15/scratch/nicer'
data_dir = 'data'
spec_dir = 'spec'


In [ ]:
os.chdir(wdir)
src_dir = f'{wdir}/{data_dir}'

obsdirs = [l.strip() for l in open('obsids2.txt').readlines()]
obsids = [o.split('/')[-2 if o[-1]=='/' else -1] for o in obsdirs]
#obsids = [o[(len(src_dir)+1):] for o in glob.glob(f'/??????????')]
print(f'There are {len(obsids)} observations')
print(', '.join(obsids))

In [ ]:
# copy data
os.system(f'mkdir -p {src_dir}')
os.chdir(src_dir)
for o in obsdirs:
    if not os.path.exists(o.split('/')[-2]):
        os.system(f'cp -r {o} .')

In [ ]:
# add geomagnetic data
os.environ['GEOMAG_PATH'] = src_dir + '/geomag'
os.environ['HEADASNOQUERY'] = ''
os.environ['HEADASPROMPT'] = '/dev/null'
if not os.path.exists(os.environ['GEOMAG_PATH']):
    subprocess.call(['nigeodown'], env=os.environ)

In [ ]:
# process the data; run in parallel
os.chdir(src_dir)
process_nicer_data(obsids, nproc=5, fresh=False)

In [ ]:

pklDir = f'{src_dir}/pkl'
os.system(f'mkdir -p {pklDir}')
os.chdir(pklDir)

# batch size so we don't do everything in one go
size_limit = 100000


# intervals for binning the PI axis
pi_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 100, 120, 150, 180, 210, 
           240, 300, 500, 750, 1000, 1100, 1200, 1400, 1501]
npi = len(pi_bins) - 1
pi_int = pd.IntervalIndex.from_breaks(pi_bins, closed='left')

tBin = 1 # Time bin value

# loop through obsids
spec = None
for iobs, obsid in enumerate(obsids):
    
    # do the work only if we haven't done it yet, 
    # i.e. when the pkl file does not exist
    pkl_file = f'spec.{obsid}.pkl'
    if not os.path.exists(pkl_file):
        
        # read the event file: a time-ordered list of events
        evtfile = f'../{obsid}/xti/event_cl/ni{obsid}_0mpu7_cl.evt'
        with fits.open(evtfile) as fp:
            tbl = Table(fp[1].data)
            timezero = fp[1].header['timezero']
        names = ['TIME', 'PI']
        tbl = tbl[names].to_pandas()
        tbl.TIME += timezero
        
        
        # extract a raw light curve so we get the good time intervals
        o = hsp.niextlc(infile=evtfile, outfile=f'tmp_{obsid}.lc', timebin=tBin, 
                    lcthresh=1.0, pirange="30:800", clobber=True)
        if o.returncode != 0:
            print(''.join(o.out))
            raise ValueError('niexlc failed')

        # read light curve
        with fits.open(f'tmp_{obsid}.lc') as fp:
            lc = Table(fp[1].data).to_pandas()
            t0 = fp[1].header['timezero']
        lc.TIME += t0
        
        # print how much data we are dealing with
        print(f'{obsid}: events-> {tbl.shape}, lc-> {len(lc)}')
        
        
        # construct a pandas time interval so we can use it with groupby
        # to construct a 2d histrogram of time or pi
        time_int = pd.IntervalIndex.from_arrays(lc.TIME-tBin/2, 
                                                lc.TIME+tBin/2, closed='left')

        # main part of counting events
        tbl2 = tbl.TIME.groupby([pd.cut(tbl.TIME, bins=time_int), 
                                 pd.cut(tbl.PI, bins=pi_int )]).count()
        # reshape it so it has axis (time, PI)
        this_spec = pd.DataFrame(tbl2.values.reshape(-1,npi), 
                                 index=lc.TIME, columns=pi_int.left)
        if spec is None:
            spec = this_spec
        else:
            spec = pd.concat([spec, this_spec])
                    

In [ ]:
spec

In [ ]:
spec.to_csv('spectra.csv')

In [ ]:
!pwd